In [1]:
from IPython.display import HTML

HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form> ''')

# Early Signs of A Future Housing Market Crash

This project aims to provide a group of cities that can be served as an early sign of a future housing market crash. We identifies cities that responded earliest during the housing market crash in 2007 by computing the time points when the median house prices are at the maximum (we call here, tipping points).  We show the cities' common characteristics. 

1. 
3. Depending on the States, e.g., *****, the median house prices of the cities are strongly correlated with the tipping point house prices during the 2007 market crash, either positively or negatively. 

In [22]:
%matplotlib inline
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import NumeralTickFormatter

output_notebook()

Loading BokehJS ...

## Data used in this project

1. The historic median house prices by city (downloaded from [here](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv)) was obtained from the [Zillow Research](http://www.zillow.com/research/data/#median-home-value).
2. The city populations of year 2010 to 2014 were obtained from census.gov data ([link](http://www.census.gov/popest/data/cities/totals/2014/files/SUB-EST2014_ALL.csv)).
3. 

In [23]:
home_data = pd.read_csv('./data/City_Zhvi_AllHomes.csv', encoding="utf-8-sig")
len_data = len(home_data)

In [24]:
# This function returns a data frame that contains tipping point house prices (column name = max_p),
# tipping point time (months since the provided time_min) (column name = months_since), 
# city size rank (SizeRank), US State name (State), and city name (RegionName).
# If 'state' is not given, all US States is considered.
# The format of the time_min and time_max is as follows.
# time_min = '2003-01'
# time_max = '2009-07'

def tipping_point(data, state, number_top_cities, time_min, time_max):
    if state != 'ALL': 
        data = data[data['State']==state].sort_values(by='SizeRank', ascending=True)
        
    # city's sizerank per state.
    city_sizerank = pd.DataFrame()
    city_sizerank['RegionName'] = data['RegionName']
    city_sizerank['SizeRank'] = data['SizeRank']
    city_sizerank['State'] = data['State']
    city_sizerank.index = data['RegionName']
    
    top_size = number_top_cities
    city_sizerank_top = city_sizerank[0:top_size]
    
    data = data.transpose()
    data.columns = data.loc['RegionName'].tolist()
    data = data.iloc[6:] # Remove the unnecessary rows
    data = data.set_index(data.index.to_datetime())
    data = data[time_min: time_max]
    data_top = data.iloc[:, 0:top_size]

    # If you want to plot time trajectories of median house prices of the cities of SizeRank <= top_size in the US,
    # uncomment the following command.
    # data_top.plot(legend=None, title=None)

    # We find the tipping point house prices and their corresponding dates (more precisely, months since time_min).
    
    max_price = data_top.max().to_frame()
    max_price.columns = ['max_p']
    max_date = data_top.idxmax().to_frame()
    max_date.columns = ['max_d']
    
    df_st = pd.concat([max_date, max_price], axis=1)
    date_begin = pd.to_datetime(time_min)

    df_st['months_since'] = (pd.to_datetime(df_st['max_d']) - date_begin).astype('timedelta64[M]')
    df_st['sizerank'] = city_sizerank_top['SizeRank']
    df_st['state'] = city_sizerank_top['State']
    df_st['city'] = city_sizerank_top['RegionName']    
    return df_st

Tipping points of the US cities of top 5000 population sizes are plotted below:

In [25]:
number_top_cities = 5000
time_min = '2003-01'
time_max = '2009-07'
# All US States are considered.
state = "ALL"
df_st_ALL = tipping_point(home_data, state, number_top_cities, time_min, time_max)
x = df_st_ALL['max_p']
y = df_st_ALL['months_since']
p = figure(title = "Tipping-points of top 5000 city (in city size)", plot_height = 300, 
           plot_width = 600, 
           x_range=(0, 2*10**6), 
           y_range=(0, 80))
p.xaxis.axis_label = 'Median House Price'
p.yaxis.axis_label = 'Months since ' + time_min 
r = p.circle(x, y, size=10, name = "foo", fill_alpha = 0.1, line_color=None)
p.xaxis.formatter = NumeralTickFormatter(format="$0,0")
show(p)

The above plot shows a triangle shape. To see if there are distinct underlying patterns for individual states, the same plot for each state is listed below.

In [26]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis3

# US states that will be analyzed
st_list = ['WA', 'CA', 'IL', 'FL', 'ND', 'MA', 'VA', 'PA', 'OR', 'CO', 'MI', 'OH']

number_top_cities = 100
time_min = '2003-01'
time_max = '2009-07'

lst = []
for st_name,i in zip(st_list, range(0, len(st_list))):
    df_st = tipping_point(home_data, st_name, number_top_cities, time_min, time_max)
    # source = ColumnDataSource(data = dict(x=x, y=y))
    p = figure(title = st_name, width = 250, height = 200,
               x_range=(0, 2*10**6), y_range=(0, 80))
    p.circle(df_st['max_p'], df_st['months_since'], size=10, fill_alpha = 0.1, line_color=None)
    p.xaxis.formatter = NumeralTickFormatter(format="$0,0")
    lst.append(p)
    
grid = gridplot( np.array(lst).reshape(4,3).tolist())
#grid = gridplot([p])
show(grid)


For example, Massachusetts, Virginia, and Pennsylvania show negative correlation between tipping point house prices and times. However, Washington shows strong correlation for cities with their median prices above $400K.

In addition, most cities in the Michigan State show that the tipping points appear early time (months since 2003-01 < 40). This indicates that Michigan cities can be served as early sign of future housing market crash if the State econo-metrics were not changed significantly compared to other States. To investigate what other cities can be considered as the early signs, we converted the data frame (pandas) containing tipping point information to the SFrame to show quickly the statistics of the tipping points corresponding to cities of "months since 2003-01" < 30.  

In [36]:
import graphlab
graphlab.canvas.set_target('ipynb')
sf = graphlab.SFrame(data = df_st_ALL) 
mask_city_early_crash = sf['months_since']<30
sf[mask_city_early_crash]['state'].show(view = 'Categorical')

The above table shows the population of the cities that can be served as the early signe of housing market crash. Interestingly, Michingan, Massachusetts, Ohio, and Indiana contain the most of the cities. We looked into more about the Michigan State. In 2007, the population of the Michigan State was even decreased from July 2005 to July 2006 ([Washington Post.com 2007]( http://www.washingtonpost.com/wp-dyn/content/article/2007/03/30/AR2007033002127.html)). To find whether the growth rate of a city population is strongly correlated with a tipping point, we retrieved the census.gov data ([link](http://www.census.gov/popest/data/cities/totals/2014/files/SUB-EST2014_ALL.csv)) and computed the city population growth rate and merged with the data frame containing the tipping point information. As shown in the figure below, there was,however, no correlation between populatin growth rate and the tipping point time when all US States are considered together (and separately; not shown below).  

In [54]:
def remove_last_word(words):
    splitted = words.split()[:-1]
    return ' '.join(splitted)

url = 'http://www.census.gov/popest/data/cities/totals/2014/files/SUB-EST2014_ALL.csv'
sf_pop = graphlab.SFrame.read_csv(url, verbose=False)

sf_pop = sf_pop[['NAME', 'STNAME', 'POPESTIMATE2010', 'POPESTIMATE2014']][1:]
sf_pop.rename({'NAME': 'city', 'STNAME':'state', 'POPESTIMATE2010':'pop2010', 'POPESTIMATE2014':'pop2014'})
# compute the average growth rate between 2010 and 2014.
sf_pop['growth_rate'] = (np.array(sf_pop['pop2014']-sf_pop['pop2010']))*100/np.array(sf_pop['pop2010'].astype(float))
# change the city name to be the same as the one used in the Zillow.com data.
sf_pop['city'] = sf_pop['city'].apply(remove_last_word)

import csv
reader = csv.reader(open('./data/states.csv', 'r'))
dict_st_to_abb = {}
for row in reader:
   k, v = row
   dict_st_to_abb[k] = v
    
sf_pop['state'] = sf_pop['state'].apply(lambda x: dict_st_to_abb[x])

# Duplicate cities are removed.
sf_pop = sf_pop.unique()
df_pop = sf_pop.to_dataframe()

df_pop = df_pop[df_pop.groupby(['city', 'state'])['growth_rate'].rank(ascending=False)==1]
sf_pop = graphlab.SFrame(df_pop)

# Two data frames obtained by processing data from census.gov and zillow.com are joined.
# And then we removed all the rows containing 'nan' or 'NaN'. 
sf = sf.join(sf_pop, on=['city', 'state'], how='inner')
sf = sf[sf['months_since'].apply(str)!='nan']
sf = sf[sf['growth_rate'].apply(str)!='nan']
sf = sf[sf['growth_rate'].apply(str)!='inf']
sf;

p = figure(title = "", width = 500, height = 400)
               # x_range=(0, 2*10**6), y_range=(0, 80))
p.circle(sf['growth_rate'], sf['months_since'], size=10, fill_alpha = 0.1, line_color=None)
p.xaxis.axis_label = 'Population Growth Rate from 2010 to 2014 (Number of People/YR)'
p.yaxis.axis_label = 'Months since ' + time_min
show(p)

In [56]:
########################
## US individual states are considered.
########################
# st_list = ['WA', 'CA', 'IL', 'FL', 'ND', 'MA', 'VA', 'PA', 'OR', 'CO', 'MI', 'OH']

# lst = []
# for st_name,i in zip(st_list, range(0, len(st_list))):
#     x = sf[sf['state']==st_name]['growth_rate']
#     y = sf[sf['state']==st_name]['months_since']
#     p = figure(title = st_name, width = 250, height = 200,x_range=(-23, 54), y_range=(0, 80))
#     p.circle(x, y, size=10, fill_alpha = 0.1, line_color=None)
#     lst.append(p)
    
# grid = gridplot( np.array(lst).reshape(4,3).tolist())
# show(grid)

In [57]:
p = figure(title = "", width = 500, height = 400, x_axis_type="log")
               # x_range=(0, 2*10**6), y_range=(0, 80))
p.circle(sf['pop2010'], sf['months_since'], size=10, fill_alpha = 0.1, line_color=None)
p.xaxis.axis_label = 'City population in 2010)'
p.yaxis.axis_label = 'Months'
show(p)

In [58]:
lst = []
for st_name,i in zip(st_list, range(0, len(st_list))):
    x = sf[sf['state']==st_name]['pop2010']
    y = sf[sf['state']==st_name]['months_since']
    p = figure(title = st_name, width = 250, height = 200, x_axis_type="log",x_range=(10**3, 10**7), y_range=(0, 80))
    p.circle(x, y, size=10, fill_alpha = 0.1, line_color=None)
    lst.append(p)
    
grid = gridplot( np.array(lst).reshape(3,4).tolist())
show(grid)

#### We will apply clusting algorithms to categorize the cities having distinct properties based on city population, its growth rate, median house price, and state. Please refer to the Gaussian Mixture Model IPython notebook (gmm.ipynb). We save the SFrame in the binary format for a quick data conversion.

In [37]:
sf.save('./data/census-zillow-sframe')

### Prediction
#### Now we are ready to evaluate the current housing market status and see if the next housing market crash is imminet or not.

In [55]:
number_top_cities = 5000
time_min = '2014-01'
time_max = '2016-10'
state = "ALL"
x, y, df_st_ALL = tipping_point(home_data, state, number_top_cities, time_min, time_max)
# source = ColumnDataSource(data = dict(x=x, y=y))
p = figure(title = "Dependency of the tipping-points on median house price per US city", plot_height = 300, 
           plot_width = 600, 
           x_range=(0, 2*10**6), 
           y_range=(0, 80))
p.xaxis.axis_label = 'Median House Price'
p.yaxis.axis_label = 'Months'
r = p.circle(x, y, 
             size=10, name = "foo", fill_alpha = 0.1, line_color=None)
#r = p.circle(graphlab.SArray(x)[mask_city_early_crash], graphlab.SArray(y)[mask_city_early_crash], 
#             size=10, name = "foo", fill_alpha = 0.1, line_color=None)
p.xaxis.formatter = NumeralTickFormatter(format="$0,0")
show(p)

In [56]:
p = figure(title = "Dependency of the tipping-points on median house price per US city", plot_height = 300, 
           plot_width = 600, 
           x_range=(0, 2*10**6), 
           y_range=(0, 80))
p.xaxis.axis_label = 'Median House Price'
p.yaxis.axis_label = 'Months'
r = p.circle(graphlab.SArray(x)[mask_city_early_crash], graphlab.SArray(y)[mask_city_early_crash], 
             size=10, name = "foo", fill_alpha = 0.1, line_color=None)
p.xaxis.formatter = NumeralTickFormatter(format="$0,0")
show(p)

In [58]:
sum(mask_city_early_crash)

250

In [74]:
for i in len(mask_city_early_crash):
    if mask_city_early_crash == True:
        

dtype: int
Rows: 5000
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... ]

In [70]:
len(y[ (y>0) & (y<25)])

1212

In [62]:
sf_y = graphlab.SFrame(y)
sf_y

X1
27.0
27.0
19.0
27.0
27.0
27.0
27.0
27.0
27.0
27.0


In [53]:
graphlab.SFrame(x)[mask_city_early_crash]

X1
257400.0
115600.0
63200.0
105200.0
52900.0
52200.0
174600.0
317100.0
663900.0
153200.0


In [ ]:
number_top_cities = 5000
time_min = '2003-01'
time_max = '2009-07'
state = "ALL"
x, y, df_st_ALL = tipping_point(home_data, 'MI', number_top_cities, time_min, time_max)

# source = ColumnDataSource(data = dict(x=x, y=y))
p = figure(title = "MI", 
           plot_height = 400, 
           plot_width = 500, 
           x_range=(0, 2*10**6), 
           y_range=(0, 80))
p.xaxis.axis_label = 'Median House Price'
p.yaxis.axis_label = 'Months'
r = p.circle(x, y, size=10, name = "foo", fill_alpha = 0.1, line_color=None)
p.xaxis.formatter = NumeralTickFormatter(format="$0,0")
show(p)

In [ ]:
# Interactive Bokeh slider

def update(median_price=50000):
    temp = df_st[df_st['max_p']>median_price]
    r.data_source.data['x'] = temp['max_p']
#    r.data_source.data['y'] = temp['months_since']
    print len(r.data_source.data['x'])
    print len(r.data_source.data['y'])
    push_notebook()

show(p)
from IPython.html.widgets import interact
interact(update, median_price=(40000, 3000000))        

In [ ]:
# subplots of 3x3, figure(1)
fig1, axes1 = plt.subplots(nrows=3, ncols=3, figsize=(15,9), sharex=True, sharey=True)
fig1.text(0.5, 0.05, 'Date (Year)', ha='center', va='center').set_fontsize(20)
fig1.text(0.05, 0.5, 'Price ($)', ha='center', va='center', rotation='vertical').set_fontsize(20)

# subplots of 3x3, figure(2)
fig2, axes2 = plt.subplots(nrows=3, ncols=3, figsize=(15,9), sharex=True, sharey=True)
fig2.text(0.5, 0.05, 'Maximum Median House Price ($)', ha='center', va='center').set_fontsize(20)
fig2.text(0.05, 0.5, 'Months after Jan 2005', ha='center', va='center', rotation='vertical').set_fontsize(20)

# subplot axes
plt_axes = [[i,j] for i in range(0,3) for j in range(0,3)]
# US states that will be analyzed
st_list = ['WA', 'CA', 'IL', 'FL', 'NJ', 'MA', 'VA', 'PA', 'OR', 'CO']

for st_name,i in zip(st_list, range(0, 9)):


    # Data preprocessing
    # Choose specific time frame when the housing marking was crashed, i.e, near 2007 december.
    # Choose the top 40 largest cities within a given state.
    state = data[data['State'] == st_name].sort_values(by='SizeRank', ascending=True)
    
    # city's sizerank per state.
    city_sizerank = pd.DataFrame()
    city_sizerank['RegionName'] = state['RegionName']
    city_sizerank['SizeRank'] = state['SizeRank']
    city_sizerank = city_sizerank.set_index('RegionName')
    
    city_sizerank_top40 = city_sizerank[0:40]
    #print city_sizerank_top40
    
    state = state.transpose()
    state.columns = state.loc['RegionName'].tolist()
    state = state.iloc[5:] # Remove the unnecessary rows
    state = state[100:160]
    state = state.set_index(state.index.to_datetime())
    state_top40 = state.iloc[:, 0:40]
    
    # We plotted time trajectories of median house prices of the top 40 cities for given states.
    state_top40.plot(ax=axes1[plt_axes[i][0], plt_axes[i][1]], legend=None, title=st_name)
    
    # Find the max house prices and their corresponding dates for the top 40 cities of each given state.
    # To show the relationship between how long the house price starts to decrease, depending on the median house price
    # of each city for a given state.
    # We plotted max house price vs. months from Jan 01, 2005 when the median house price was peaked.
    
    max_price = state_top40.max().to_frame()
    max_price.columns = ['max_p']
    max_date = state_top40.idxmax().to_frame()
    max_date.columns = ['max_d']
    # print max_date_price.loc[st_name]
    #print pd.concat([max_date, max_price], axis=1)
    max_date_price.loc[st_name][0] = pd.concat([max_date, max_price], axis=1)
    df_st = max_date_price.loc[st_name][0]
    date_begin = pd.to_datetime('2005-01')
    df_st['days_since'] = (pd.to_datetime(df_st['max_d']) - date_begin).astype('timedelta64[M]')
    df_st['sizerank'] = city_sizerank_top40['SizeRank']
    
    df_st.plot(ax=axes2[plt_axes[i][0], plt_axes[i][1]], x='max_p', y='days_since', marker='o', 
               kind='scatter', title=st_name, legend=None, c = df_st['sizerank']/len_data, cmap = 'rainbow')
    
    #print city_sizerank_top40
 
## Rank of each data point is shown in the scatter plot.
#     for item in zip(df_st['max_p'], df_st['days_since'], df_st['sizerank']):
#         if math.isnan(item[0]) == False and math.isnan(item[1]) == False:
#             axes2[plt_axes[i][0], plt_axes[i][1]].text(item[0], item[1], str(item[2]))
    
    # We did not consider the cases of the house median prices lower than $100K.
    axes2[plt_axes[i][0],plt_axes[i][1]].set_xlim([100000,1000000])
    axes2[plt_axes[i][0], plt_axes[i][1]].set_ylim([0, 40])
    axes2[plt_axes[i][0], plt_axes[i][1]].set_xlabel('')
    axes2[plt_axes[i][0], plt_axes[i][1]].set_ylabel('')
    axes2[plt_axes[i][0], plt_axes[i][1]].set_xticks(np.arange(100000,1000000, 250000))

In [ ]:
print len_data

In [ ]:
#plt.plot([1,2,3,4,5], [1,2,3,4,5], c=np.linspace(0,1,5), kind='scatter')
plt.scatter([1,2,3,4,5], [1,2,3,4,5], c=np.linspace(0,10,5), cmap='rainbow', s=50)

In [ ]:
np.linspace(0,1,10)

In [17]:
HTML(''' <script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#999; background:#fff;">
</footer>''')